In [110]:
import networkx as nx
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
import numpy as np
import math
import pickle
from tqdm import tqdm_notebook
from multiprocessing import Pool
import math
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

In [111]:
#
# Parameter Settings
# --------------------------------------------------------------------------------------------------------------------------
latent_dim = 300
epochs = 300
seq_length = 8
lr = 1e-6
param_lambda = 0.2
Z = 1e5
min_num_nodes = 6
weight_norm_type = 'log'  # [log, min-max]
offset = 3

## Data Loading and Graph Constructing

In [112]:
CustInfo = pd.read_csv('../../AMLAML/customerinformation.csv')
CustInfo['open_date'] = pd.to_datetime(CustInfo.Open_Date)

SARCase = pd.read_csv('../../AMLAML/sarcase.csv')
SARCase['created_date'] = pd.to_datetime(SARCase.Created_Date)

WireTrans = pd.read_csv('../data/new_wire.csv')
WireTrans['trans_date'] = pd.to_datetime(WireTrans.trandt)
WireTrans.head()

,Unnamed: 0,Acct_Txn_Amt,acctrxncurrencycode,Agent_Id,appl,As_Of_Date,Bank_Country_Code,bnf_id,bnfbankcountrycode,bnf,...,Account_Nbr_10,Account_Nbr_11,customerno,customer_to,customer_from,customer_to2,customer_from2,trans_date,org_cust,bnf_cust
0,0,345000.000,USD,nan,SAV,06JUN2019,HK,534718887838,HK,NaN,...,DgUFAQUCBnQKBCM=,DgUFAQUCBnQKBCM=,BgcDAgcBIw==,AgYFBwQKDnwOCQQj,DgUFAQUDBHYLAwQj,extra_vertex,extra_vertex,2017-01-25,BgcDAgcBIw==,AgYFBwQKDnwOCQQj
1,1,5647.000,USD,nan,SAV,06JUN2019,HK,904138007505,NaN,NaN,...,DgUFAQYKBnQOBCM=,DgUFAQYKBnQOBCM=,DwUBBwACIw==,DwUBBwADIw==,fnACBnd9cAUACAcj,DwUBBwADIw==,extra_vertex,2017-01-04,fnACBnd9cAUACAcj,DwUBBwACIw==
2,2,4450000.000,USD,nan,SAV,06JUN2019,HK,656157759287,CN,NaN,...,DgUFAQYHBnUBASM=,DgUFAQYHBnUBASM=,AgUACAUAIw==,AQAHAQAFAXEAAw8j,AgUACAUHIw==,extra_vertex,AgUACAUHIw==,2017-01-13,AgUACAUAIw==,AQAHAQAFAXEAAw8j
3,3,19090.050,EUR,nan,SAV,06JUN2019,HK,360166300,DE,NaN,...,DgUFAQYHBncMBiM=,DgUFAQYHBncMBiM=,AgUCBQIAIw==,BAMBAQMEBXQJIw==,AgUCBQIFIw==,extra_vertex,AgUCBQIFIw==,2017-01-19,AgUCBQIAIw==,BAMBAQMEBXQJIw==
4,4,238369.080,USD,nan,SAV,06JUN2019,HK,904133012003,NaN,NaN,...,DgUFAQYBBnULASM=,DgUFAQYBBnULASM=,BAUAAgUCIw==,BAUAAgUCIw==,DwMAAQ0BBXMLCCM=,BAUAAgUCIw==,extra_vertex,2017-01-16,DwMAAQ0BBXMLCCM=,BAUAAgUCIw==


## ```WIRE_AMTIN``` & ```WIRE_AMTOT``` Normalization
```Acct_Txn_Amt``` is recorded in terms of the currency of that transaction.
```WIRE_AMTOT``` and ```WIRE_AMTIN``` are recorded in terms of the same currency via currency conversion.
Owing to the lack of the balance, account-wise normalization methods cannot be applied.
The applied normalization methods are listed as follows:
+ log normalization
+ min-max normalization

In [113]:
# Log normalization
def weight_normalize(df, col_wire_in='WIRE_AMTIN', col_wire_out='WIRE_AMTOT', weight_norm_type='log'):
    if weight_norm_type == 'log':
        df["norm_wire_amt_out"] = df.apply(lambda x: math.log(x[col_wire_out]+1), axis=1)
        df["norm_wire_amt_in"] = df.apply(lambda x: math.log(x[col_wire_in]+1), axis=1)
        df["wire_amt"] = df.apply(lambda x: x[col_wire_in]+x[col_wire_out], axis=1)
        df["norm_wire_amt"] = df.apply(lambda x: math.log(x["wire_amt"]+1), axis=1)
    return df

## Aggregation of values
In transaction data, we want to aggregate transaction values in the past few months by using ```groupby()``` and ```sum()``` functions.

In [114]:
# Convert to undirected vertex index 
def convert_undirected_vertex_index(df, col_from_vertex='org_cust', col_to_vertex='bnf_cust'):
    df['vertex_index'] = df.apply(lambda x: str(set([x[col_from_vertex], x[col_to_vertex]])), axis=1)
    df_new = df.groupby('vertex_index').sum().reset_index()
    df_new[col_from_vertex] = df_new.apply(lambda x: x['vertex_index'][1:-1].split(',')[0][1:-1], axis=1)
    df_new[col_to_vertex] = df_new.apply(lambda x: x['vertex_index'][1:-1].split(',')[1][2:-1], axis=1)
    return df_new

In [115]:
# Convert to undirected vertex index 
def group_trans_by_count_larger(df, thres=8000, 
                                col_from_vertex='org_cust', col_to_vertex='bnf_cust', 
                                col_wire_in='WIRE_AMTIN', col_wire_out='WIRE_AMTOT'):
    df['vertex_index'] = df.apply(lambda x: str(set([x[col_from_vertex], x[col_to_vertex]])), axis=1)
    df['norm_wire_amt'] = df.apply(lambda x: 1 if x[col_wire_out] > thres or x[col_wire_in] > thres else 0, axis=1)
    df_new = df.groupby('vertex_index').sum().reset_index()
    df_new[col_from_vertex] = df_new.apply(lambda x: x['vertex_index'][1:-1].split(',')[0][1:-1], axis=1)
    df_new[col_to_vertex] = df_new.apply(lambda x: x['vertex_index'][1:-1].split(',')[1][2:-1], axis=1)
    return df_new

## Fetch data by date

In [116]:
class DateYM:
    def __init__(self, year, month):
        self.year = year
        self.month = month - 1    # 0 ~ 11, from Jan to Dec
        
    def export_tuple(self):
        return (self.year, self.month+1)
    
    def add_year(self, y):
        self.year += y
        
    def substract_year(self, y):
        self.year -= y
        
    def add_month(self, m):
        self.month += m
        self.year += math.floor(self.month / 12)
        self.month = self.month % 12
        
    def subtract_month(self, m):
        self.month -= m
        tmp_year = math.floor(self.month / 12)
        self.year += tmp_year
        self.month += -tmp_year * 12
        
    def is_larger_than(self, ym):
        return self.year*12 + self.month > ym.year*12 + ym.month
    
    def is_smaller_than(slef, ym):
        return self.year*12 + self.month < ym.year*12 + ym.month
    
    def is_equal(self, ym):
        return self.year*12 + self.month == ym.year*12 + ym.month

    
def list_date_tuples(from_date, to_date):
    ret = []
    tmp_date = DateYM(*from_date.export_tuple())
    while not tmp_date.is_larger_than(to_date):
        ret.append(tmp_date.export_tuple())
        tmp_date.add_month(1)
    return ret


def fetch_data_by_month(date_ym, trans_offset=6):
    year, month = date_ym
    
    # Get view: WireTrans
    from_date = pd.to_datetime("{}/{}/{}".format(month, 1, year))
    to_date = from_date + pd.DateOffset(months=1)
    offset_date = from_date - pd.DateOffset(months=trans_offset)
    view_wiretrans = WireTrans[(WireTrans.trans_date > offset_date) & 
                               (WireTrans.trans_date < to_date)]
    
    # Get view: CustInfo
    view_customer = CustInfo[CustInfo.open_date < to_date]
    
    # Attach label onto CustInfo
    target_list = SARCase[(SARCase.Status_SAR == 4) & 
                          (SARCase.created_date > from_date) & 
                          (SARCase.created_date < to_date)]['customerno'].unique()
    # print ('# of SAR customers: {}'.format(len(target_list)))
    view_customer['label'] = view_customer.apply(lambda x: 1 if x['customerno'] in target_list else 0, axis=1)
    
    return view_wiretrans, view_customer

In [117]:
#
# Find all nodes across the squential graphs
# --------------------------------------------------------------------------------------------------------------------------
list_nodes_key = list(set(WireTrans.org_cust.values) | set(WireTrans.bnf_cust.values))
dict_nodes_key2int = dict(zip(list_nodes_key, range(len(list_nodes_key))))
dict_nodes_int2key = dict(zip(range(len(list_nodes_key)), list_nodes_key))
with open('dict_nodes_key2int.pickle', 'wb') as f:
    pickle.dump(dict_nodes_key2int, f)
with open('dict_nodes_int2key.pickle', 'wb') as f:
    pickle.dump(dict_nodes_int2key, f)

## Compute Eigenmaps of a Weighted Graph Laplacian

In [118]:
def eigen_maps(G, min_num_nodes=5, weight_attr='norm_wire_amt'):
    """ Split graph G into connected subgraph, and then
        compute SA w.r.t. each subgraph respectively.
    """
    # Get all the connected components and their graph Laplacians
    print('  |-- Get all the connected components ...')
    subVs = list(nx.connected_components(G))
    subGs = [G.subgraph(subV) for subV in subVs]
    subLs = [nx.normalized_laplacian_matrix(subG, weight=weight_attr) for subG in subGs]

    # Initialize the vertex embedding with one-hot indexing of components
    print('  |-- Initialize the vertex embedding ...')
    embedding = np.zeros([G.number_of_nodes(), len(subVs)])
    for index in range(len(subVs)):
        embedding[list(subVs[index]), index] = 1

    # Do spectral analysis respectively
    print('  |-- Graph Processing ...')
    for index in range(len(subGs)):
        subV2V = list(subGs[index].nodes())  # ordered index in terms of V of subV
        if len(subV2V) < min_num_nodes:
            continue
        subL = subLs[index]
        
        # Compute eigenmaps
        is_complete_eigen = False
        while not is_complete_eigen:
            try:
                eigval_subL, eigvec_subL = scipy.sparse.linalg.eigs(subL, k=min(6, len(subV2V)-2), which="SR")
                is_complete_eigen = True
            except:
                print("Re-run eigenmaps ...")
        eigval_subL = np.real(eigval_subL)
        sorted_index = np.argsort(eigval_subL)
        eigval_subL = eigval_subL[sorted_index]
        eigvec_subL = np.real(eigvec_subL)[:,sorted_index]

        # Discard not important (i.e., zero and large) eigvectors
        tmp_vec = eigvec_subL[:, (eigval_subL > 0) * (eigval_subL < 0.6)]
        if tmp_vec.shape[1] == 0:
            continue
        # Conpute the vertex embeeding w.r.t. to vertices in $new_subV
        for i in list(range(tmp_vec.shape[1]-1, -1, -1)):
            if np.sum(tmp_vec[:,i]>0)>0 and np.sum(tmp_vec[:,i]<0)>0:
                tmp_vec = np.insert(tmp_vec, i, np.sign(tmp_vec[:,i]), axis=1)
                tmp_vec[:, i+1] = np.absolute(tmp_vec[:, i+1])

        tmp_embedding = np.zeros([G.number_of_nodes(), tmp_vec.shape[1]])
        tmp_embedding[subV2V, :] = tmp_vec
        embedding = np.append(embedding, tmp_embedding, axis=1)
        # print("subG index = {}, subL.shape = {}, tmp_embedding.shape={}".format(index, subL.shape, tmp_embedding.shape))
        
    return embedding

In [119]:
from_date_ym = DateYM(2017, 1)
to_date_ym = DateYM(2019, 8)

list_date_seq = list_date_tuples(from_date_ym, to_date_ym)
projectors_shape = {}
for i, date_ym in zip(range(len(list_date_seq)), list_date_seq):
    print("Processing the data in {}".format(date_ym))
    
    # Fetch data by month and aggregate values by column
    view_wiretrans, view_customer = fetch_data_by_month(date_ym, trans_offset=offset)
    view_wiretrans = convert_undirected_vertex_index(view_wiretrans)
    view_wiretrans = weight_normalize(view_wiretrans)
    
    # Construct graph
    G_t = nx.from_pandas_edgelist(view_wiretrans, 'org_cust', 'bnf_cust', edge_attr='norm_wire_amt')
    dict_sub_nodes_key2int = dict(zip(list(G_t.nodes()), range(G_t.number_of_nodes())))
    dict_sub_nodes_int2key = dict(zip(range(G_t.number_of_nodes()), list(G_t.nodes())))
    
    G_t = nx.relabel_nodes(G_t, dict_sub_nodes_key2int)
    G_embedding = eigen_maps(G_t, min_num_nodes=min_num_nodes)
    
    # Write eigenmap into a file
    eigen_path = "eigen/offset_{}/aggregate/weight_{}_eigenmap_{}-{}".format(
        offset, weight_norm_type, date_ym[0], date_ym[1])
    np.savez_compressed(eigen_path, G_embedding)
    # np.save(eigen_path, G_embedding)
    
    # Write dict int2key / key2int into files
    key2int_path = "offset_{}/aggregate/weight_{}_nodes_key2int_{}-{}.pickle".format(
        offset, weight_norm_type, date_ym[0], date_ym[1])
    with open(key2int_path, 'wb') as f:
        pickle.dump(dict_sub_nodes_key2int, f)
    int2key_path = "offset_{}/aggregate/weight_{}_nodes_int2key_{}-{}.pickle".format(
        offset, weight_norm_type, date_ym[0], date_ym[1])
    with open(int2key_path, 'wb') as f:
        pickle.dump(dict_sub_nodes_int2key, f)
 
    projectors_shape[date_ym] = (G_embedding.shape[1], latent_dim)
    print("SubGraph {}: has {} vertices, {} edges; Embedding shape: {}".
          format(date_ym, G_t.number_of_nodes(), G_t.number_of_edges(), G_embedding.shape))

Processing the data in (2017, 1)
  |-- Get all the connected components ...
  |-- Initialize the vertex embedding ...
  |-- Graph Processing ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
SubGraph (2017, 1): has 27469 vertices, 26780 edges; Embedding shape: (27469, 3779)
Processing the data in (2017, 2)
  |-- Get all the connected components ...
  |-- Initialize the vertex embedding ...
  |-- Graph Processing ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
SubGraph (2017, 2): has 38660 vertices

  |-- Get all the connected components ...
  |-- Initialize the vertex embedding ...
  |-- Graph Processing ...
Re-run eigenmaps ...
Re-run eigenmaps ...
SubGraph (2019, 4): has 57278 vertices, 62341 edges; Embedding shape: (57278, 5062)
Processing the data in (2019, 5)
  |-- Get all the connected components ...
  |-- Initialize the vertex embedding ...
  |-- Graph Processing ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
Re-run eigenmaps ...
SubGraph (2019, 5): has 53995 vertices, 58348 edges; Embedding shape: (53995, 4977)
Processing the data in (2019, 6)
  |-- Get all the connected components ...
  |-- Initialize the vertex embedding ...
  |-- Graph Processing ...
Re-run eigenmaps ...
SubGraph (2019, 6): has 55511 vertices, 59899 edges; Embedding shape: (55511, 4975)
Processing the data in (2019, 7)
  |-- Get all the connected components ...
  |-- Initialize the vertex embedding ...
  |-- Graph Processing ...
Re-run eigenmaps ...
SubGraph (2019, 7): has 55029 ver

In [120]:
from_date_ym = DateYM(2018, 1)
to_date_ym = DateYM(2019, 6)
list_date_seq = list_date_tuples(from_date_ym, to_date_ym)

In [121]:
with open('eigen/offset_{}/aggregate/weight_{}_projectors_shape'.format(offset, weight_norm_type), 'wb') as f:
    pickle.dump(projectors_shape, f)

4 steps to prepare files:
+ Read sequential graphs;
+ Convert to symmetric graph Laplacian;
+ Spectral analysis;
+ Write the eigenmaps into files.

## Embedding Projection on a Dynamic graph

In [122]:
def row_normalize(M):
    S = scipy.array(M.sum(axis=1)).flatten()
    S[S != 0] = 1.0 / S[S != 0]
    Q = scipy.sparse.spdiags(S.T, 0, *M.shape, format='csr')
    return Q * M

In [123]:
def matrix_index_map(from_matrix, from_int2key, to_key2int):
    """Map matrix to another one by row-index keys
    """
    from_keys = map(lambda x: from_int2key[x], list(range(from_matrix.shape[0])))
    to_indices = list(map(lambda x: to_key2int[x] if x in to_key2int.keys() else -1, from_keys))
    ret = np.zeros((len(to_key2int), from_matrix.shape[1]))
    for from_row_index in range(from_matrix.shape[0]):
        to_row_index = to_indices[from_row_index]
        if to_row_index == -1:
            continue
        ret[to_row_index, :] = from_matrix[from_row_index, :]
    return ret

In [124]:
def update_params(t):
    
    date_ym = list_date_seq[t]

    # Fetch data by month and aggregate values by column
    view_wiretrans, view_customer = fetch_data_by_month(date_ym, trans_offset=offset)
    view_wiretrans = convert_undirected_vertex_index(view_wiretrans)
    view_wiretrans = weight_normalize(view_wiretrans)
    
    # Construct graph
    G_t = nx.from_pandas_edgelist(view_wiretrans, 'org_cust', 'bnf_cust', edge_attr='norm_wire_amt')

    E_t = np.load('eigen/offset_{}/aggregate/weight_{}_eigenmap_{}-{}.npz'.format(
        offset, weight_norm_type, date_ym[0], date_ym[1]))['arr_0']
    F_t = projectors[date_ym]
    key2int_t = dict_key2int[date_ym]
    int2key_t = dict_int2key[date_ym]
    G_t = nx.relabel_nodes(G_t, key2int_t)    
    A_t = row_normalize(nx.to_scipy_sparse_matrix(G_t))

    t_plus = t + 1
    if t_plus < seq_length:
        date_ym_plus = list_date_seq[t_plus]
        E_t_plus = np.load('eigen/offset_{}/aggregate/weight_{}_eigenmap_{}-{}.npz'.format(
            offset, weight_norm_type, date_ym_plus[0], date_ym_plus[1]))['arr_0']
        F_t_plus = projectors[date_ym_plus]
        key2int_t_plus = dict_key2int[date_ym_plus]
        int2key_t_plus = dict_int2key[date_ym_plus]
    else:
        E_t_plus = None
        F_t_plus = None
        key2int_t_plus = None
        int2key_t_plus = None

    t_minus = t - 1
    if t_minus > -1:
        date_ym_minus = list_date_seq[t_minus]
        E_t_minus = np.load('eigen/offset_{}/aggregate/weight_{}_eigenmap_{}-{}.npz'.format(
            offset, weight_norm_type, date_ym_minus[0], date_ym_minus[1]))['arr_0']
        F_t_minus = projectors[date_ym_minus]
        key2int_t_minus = dict_key2int[date_ym_minus]
        int2key_t_minus = dict_int2key[date_ym_minus]
    else:
        E_t_minus = None
        F_t_minus = None
        key2int_t_minus = None
        int2key_t_minus = None

    for itr in range(3):
        tran = A_t * E_t
        emb = np.dot(E_t, F_t)
        tmp = np.sum(F_t ** 2) - Z
        if F_t_plus is None:
            tmp2 = matrix_index_map(from_matrix=np.dot(E_t_minus, F_t_minus), 
                                    from_int2key=int2key_t_minus, 
                                    to_key2int=key2int_t)
            grad_F_t = np.dot((E_t - tran).T, (emb - np.dot(tran, F_t))) + \
                       np.dot(E_t.T, (emb - tmp2)) + \
                       param_lambda * tmp * F_t
        elif F_t_minus is None:
            tmp2 = matrix_index_map(from_matrix=np.dot(E_t_plus, F_t_plus),
                                    from_int2key=int2key_t_plus, 
                                    to_key2int=key2int_t)
            grad_F_t = np.dot((E_t - tran).T, (emb - np.dot(tran, F_t))) + \
                       np.dot(E_t.T, (emb - tmp2)) + \
                       param_lambda * tmp * F_t
        else:
            tmp2 = matrix_index_map(from_matrix=np.dot(E_t_minus, F_t_minus), 
                                    from_int2key=int2key_t_minus, 
                                    to_key2int=key2int_t)
            tmp3 = matrix_index_map(from_matrix=np.dot(E_t_plus, F_t_plus),
                                    from_int2key=int2key_t_plus, 
                                    to_key2int=key2int_t)
            grad_F_t = np.dot((E_t - tran).T, (emb - np.dot(tran, F_t))) + \
                       np.dot(E_t.T, (2 * emb - tmp2 - tmp3)) + \
                       param_lambda * tmp * F_t
        F_t -= lr * grad_F_t

        if F_t_plus is not None:
            tran = A_t * matrix_index_map(from_matrix=E_t_plus, 
                                          from_int2key=int2key_t_plus, 
                                          to_key2int=key2int_t)
            tran = matrix_index_map(from_matrix=tran, 
                                    from_int2key=int2key_t, 
                                    to_key2int=key2int_t_plus)
            emb = np.dot(E_t_plus, F_t_plus)
            tmp = np.sum(F_t_plus ** 2) - Z
            tmp2 = matrix_index_map(from_matrix=np.dot(E_t, F_t), 
                                    from_int2key=int2key_t, 
                                    to_key2int=key2int_t_plus)
            grad_F_t_plus = np.dot((E_t_plus - tran).T, (emb - np.dot(tran, F_t_plus))) + \
                            np.dot(E_t_plus.T, (emb - tmp2)) + \
                            param_lambda * tmp * F_t_plus
            F_t_plus -= lr * grad_F_t_plus

        if F_t_minus is not None:
            tran = A_t * matrix_index_map(from_matrix=E_t_minus, 
                                          from_int2key=int2key_t_minus, 
                                          to_key2int=key2int_t)
            tran = matrix_index_map(from_matrix=tran, 
                                    from_int2key=int2key_t, 
                                    to_key2int=key2int_t_minus)
            emb = np.dot(E_t_minus, F_t_minus)
            tmp = np.sum(F_t_minus ** 2) - Z
            tmp2 = matrix_index_map(from_matrix=np.dot(E_t, F_t), 
                                    from_int2key=int2key_t, 
                                    to_key2int=key2int_t_minus)
            grad_F_t_minus = np.dot((E_t_minus - tran).T, (emb - np.dot(tran, F_t_minus))) + \
                             np.dot(E_t_minus.T, (emb - tmp2)) + \
                             param_lambda * tmp * F_t_minus
            F_t_minus -= lr * grad_F_t_minus

    projectors[date_ym] = F_t
    if F_t_plus is not None:
        projectors[date_ym_plus] = F_t_plus
    if F_t_minus is not None:
        projectors[date_ym_minus] = F_t_minus
    
    return (date_ym, F_t, F_t_plus, F_t_minus)

In [126]:
def validate(t):
    
    date_ym = list_date_seq[t]

    # Fetch data by month and aggregate values by column
    view_wiretrans, view_customer = fetch_data_by_month(date_ym, trans_offset=offset)
    view_wiretrans = convert_undirected_vertex_index(view_wiretrans)
    view_wiretrans = weight_normalize(view_wiretrans)
    
    # Construct graph
    G_t = nx.from_pandas_edgelist(view_wiretrans, 'org_cust', 'bnf_cust', edge_attr='norm_wire_amt')

    E_t = np.load('eigen/offset_{}/aggregate/weight_{}_eigenmap_{}-{}.npz'.format(
        offset, weight_norm_type, date_ym[0], date_ym[1]))['arr_0']
    F_t = projectors[date_ym]
    emb_t = np.dot(E_t, F_t)
    key2int_t = dict_key2int[date_ym]
    G_t = nx.relabel_nodes(G_t, key2int_t)    
    A_t = row_normalize(nx.to_scipy_sparse_matrix(G_t))

    t_plus = t + 1
    if t_plus < seq_length:
        date_ym_plus = list_date_seq[t_plus]
        E_t_plus = np.load('eigen/offset_{}/aggregate/weight_{}_eigenmap_{}-{}.npz'.format(
            offset, weight_norm_type, date_ym_plus[0], date_ym_plus[1]))['arr_0']
        F_t_plus = projectors[date_ym_plus]
        emb_t_plus = np.dot(E_t_plus, F_t_plus)
        int2key_t_plus = dict_int2key[date_ym_plus]
    else:
        E_t_plus = None
        F_t_plus = None
        emb_t_plus = None

    t_minus = t - 1
    if t_minus > -1:
        date_ym_minus = list_date_seq[t_minus]
        E_t_minus = np.load('eigen/offset_{}/aggregate/weight_{}_eigenmap_{}-{}.npz'.format(
            offset, weight_norm_type, date_ym_minus[0], date_ym_minus[1]))['arr_0']
        F_t_minus = projectors[date_ym_minus]
        emb_t_minus = np.dot(E_t_minus, F_t_minus)
        int2key_t_minus = dict_int2key[date_ym_minus]
    else:
        E_t_minus = None
        F_t_minus = None
        emb_t_minus = None       

    loss = 0.5 * np.sum((emb_t - A_t * emb_t) ** 2) + param_lambda * (np.sum(F_t ** 2) - Z) ** 2
    if t_plus < seq_length:
        tmp = matrix_index_map(from_matrix=emb_t_plus, 
                               from_int2key=int2key_t_plus, 
                               to_key2int=key2int_t)
        loss += 0.5 * np.sum((emb_t - tmp) ** 2)
    if t_minus > -1:
        tmp = matrix_index_map(from_matrix=emb_t_minus, 
                               from_int2key=int2key_t_minus, 
                               to_key2int=key2int_t)
        loss += 0.5 * np.sum((emb_t - tmp) ** 2)
    return loss

In [125]:
projectors = {}
for i in list_date_seq:
    projectors[i] = np.random.rand(*projectors_shape[i])

In [127]:
dict_int2key = {}
dict_key2int = {}
for date_ym in list_date_seq:
    with open("offset_{}/aggregate/weight_{}_nodes_int2key_{}-{}.pickle".format(
        offset, weight_norm_type, date_ym[0], date_ym[1]), 'rb') as f:
        int2key_t = pickle.load(f)
        dict_int2key[date_ym] = int2key_t
    with open("offset_{}/aggregate/weight_{}_nodes_key2int_{}-{}.pickle".format(
        offset, weight_norm_type, date_ym[0], date_ym[1]), 'rb') as f:
        key2int_t = pickle.load(f)
        dict_key2int[date_ym] = key2int_t

In [128]:
seq_length = len(list_date_seq)
num_cores = 16
print("Training ......")
for epoch in range(epochs):        
    #
    # Projectors training
    # -------------------------------------------------------------------------------------------------------------------
    res_list = []
    with Pool(processes=num_cores) as p:
        max_ = seq_length
        with tqdm_notebook(total=max_) as pbar:
            for t, res in tqdm_notebook(enumerate(p.imap_unordered(update_params, range(seq_length)))):
                pbar.update()
                res_list.append(res)
    
    for res in res_list:
        key = res[0]
        value = res[1]
        projectors[key] = value
    
    #
    # Validation
    # -------------------------------------------------------------------------------------------------------------------
    res_list = []
    with Pool(processes=num_cores) as p:
        max_ = seq_length
        with tqdm_notebook(total=max_) as pbar:
            for t, res in tqdm_notebook(enumerate(p.imap_unordered(validate, range(seq_length)))):
                pbar.update()
                res_list.append(res)
        
    # Save the checkpoint
    for t in range(seq_length):
        date_ym = list_date_seq[t]
        np.save('model/offset_{}/aggregate/weight_{}_projctr_{}-{}'.format(
            offset, weight_norm_type, date_ym[0], date_ym[1]), projectors[date_ym])  
    print("Epoch {} validation = {}".format(epoch, np.mean(res_list)))

Training ......


Epoch 0 validation = 10762026375.721786


Epoch 1 validation = 5026836712.405437


Epoch 2 validation = 2745114721.99706


Epoch 3 validation = 1641786063.418553


...

Epoch 295 validation = 320611.33080828056


Epoch 296 validation = 320546.7001336998


Epoch 297 validation = 320483.8884073368


Epoch 298 validation = 320421.19432751794


Epoch 299 validation = 320358.5175641457


In [107]:
offset

6